In [15]:
import numpy as np
import matplotlib.pyplot as plt
from layers import *
from gradientCheck import *
from relativeError import *
%reload_ext autoreload
%autoreload 2


## 看affine_forward是否实现正确

In [21]:
num_inputs = 2
input_shape = (4, 5, 6)
output_dim = 3

input_size = num_inputs * np.prod(input_shape)
weight_size = output_dim * np.prod(input_shape)

x = np.linspace(-0.1, 0.5, num=input_size).reshape(num_inputs, *input_shape)
w = np.linspace(-0.2, 0.3, num=weight_size).reshape(np.prod(input_shape), output_dim)
b = np.linspace(-0.3, 0.1, num=output_dim)

out, _ = affine_forward(x, w, b)
correct_out = np.array([[ 1.49834967,  1.70660132,  1.91485297],
                        [ 3.25553199,  3.5141327,   3.77273342]])

# Compare your output with ours. The error should be around 1e-9.
print('Testing affine_forward function:')
print('difference: ',relative_error(out, correct_out))

Testing affine_forward function:
('difference: ', 9.7698477288066354e-10)


## 梯度检查： 看affine_backward是否实现正确

In [20]:
np.random.seed(231)

X = np.random.rand(60).reshape(3, 4, 5)

W = np.random.randn(20, 5)

b = np.random.rand(5)

out, cache = affine_forward(X, W, b)

dout = 10 * np.random.randn(*out.shape)

f_X = lambda x: affine_forward(x, W, b)[0]

f_W = lambda w: affine_forward(X, w, b)[0]

f_b = lambda b: affine_forward(X, W, b)[0]

dX_num = numerical_gradient(f_X, X, dout)

dW_num = numerical_gradient(f_W, W, dout)

db_num = numerical_gradient(f_b, b, dout)
print(np.sum(dX_num))

dX, dW, db = affine_backward(dout, cache)

print(np.sum(dX))


print('dX relative error: ',relative_error(dX, dX_num))

print('dW relative error: ',relative_error(dW, dW_num))

print('db relative error: ',relative_error(db, db_num))

162.720377666
162.720377669
('dX relative error: ', 4.4992335547692555e-10)
('dW relative error: ', 1.38140535474486e-09)
('db relative error: ', 2.7271785292348208e-11)


## 梯度检查： 看relu_backward是否实现正确

In [23]:
X = np.random.randn(3, 4)

print(X)

out, cache = relu_forward(X)

dout = np.random.randn(*out.shape)

dX = relu_backward(dout, cache)


f = lambda x: relu_forward(x)[0]

dX_num = numerical_gradient(f, X, dout)

print('dX relative error: ',relative_error(dX, dX_num))

[[-1.16098537 -1.33673751  0.67989698  0.70258129]
 [ 0.03928057 -0.09810845  0.21523806 -0.33828224]
 [-1.33485215 -1.16741677 -0.49441246 -0.55104268]]
('dX relative error: ', 2.2755828242956136e-12)


## 梯度检查： 看affine_relu_backward是否实现正确

In [24]:
np.random.seed(231)

X = np.random.rand(60).reshape(3, 4, 5)

W = np.random.randn(20, 5)

b = np.random.rand(5)

out, cache = affine_relu_forward(X, W, b)

dout = 10 * np.random.randn(*out.shape)

f_X = lambda x: affine_relu_forward(x, W, b)[0]

f_W = lambda w: affine_relu_forward(X, w, b)[0]

f_b = lambda b: affine_relu_forward(X, W, b)[0]

dX_num = numerical_gradient(f_X, X, dout)

dW_num = numerical_gradient(f_W, W, dout)

db_num = numerical_gradient(f_b, b, dout)
print(np.sum(dX_num))

dX, dW, db = affine_relu_backward(dout, cache)

print(np.sum(dX))


print('dX relative error: ',(relative_error(dX, dX_num)))

print('dW relative error: ',(relative_error(dW, dW_num)))

print('db relative error: ',(relative_error(db, db_num)))

-1.77546717238
-1.77546717233
('dX relative error: ', 8.2209610185603213e-11)
('dW relative error: ', 2.0229685915422614e-10)
('db relative error: ', 5.0005460559918315e-13)


In [48]:
scores = np.random.randn(50, 6)

y = np.random.randint(6, size=(50, ))

loss, dscores = softmax_loss(scores, y)

loss_true, dscores_true = softmax_loss_true(scores, y)

dout = 1

f_sc = lambda scores: softmax_loss_true(scores, y)[0]

dscores_num = numerical_gradient(f_sc, scores, dout)

dscores_num2 = eval_numerical_gradient(f_sc, scores, verbose=False)

print('dscores relative error: ',(relative_error(dscores_true, dscores)))


('dscores relative error: ', 5.5968326050866418e-16)


## 需要实现一个任意层神经网络模型类

## 接下来实现公用的训练器

In [49]:
'W%d'%2

'W2'

In [50]:
np.random.rand(3)

array([ 0.98222103,  0.20244692,  0.38018709])

In [51]:
reversed(np.arange(5))

In [52]:
range(5)

[0, 1, 2, 3, 4]

In [53]:
reversed(range(5))

In [54]:
for i in reversed(np.arange(5)):
    print i

4
3
2
1
0


In [55]:
for i in reversed(range(3)):
    print i

2
1
0
